In [2]:
import json
import os
import pandas as pd


In [26]:
path_py = "../ODE-RNN/Résultats/"
path_r = "../../data/synthetic_bph_1/Résultats R script/"

In [29]:
res_py = pd.DataFrame(columns=['Random effect', 'Model', 'nb_epochs', "MAE moyenne sur l'entrainement", "MAE moyenne sur le test"])
for file in os.listdir(path_py):
    df = pd.read_json(path_py + file)
    res_py = pd.concat((res_py,df))
res_py = res_py.sort_values(by="MAE moyenne sur l'entrainement")
res_py 

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_9404\750061948.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  res_py = pd.concat((res_py,df))


,Random effect,Model,nb_epochs,MAE moyenne sur l'entrainement,MAE moyenne sur le test
0,False,RNN,42680,0.401300,0.630023
0,False,LSTM,76514,0.681355,1.022089
0,True,RNN,11942,1.474785,2.827599
0,False,LSTM,14497,2.216798,4.184069
0,True,LSTM,4924,9.264661,28.547917


In [28]:
res_r = pd.read_csv(path_r + "Performances_moyennes.csv")
res_r

,Unnamed: 0,MAE_train,MAE_test,MSE_train,MSE_test
0,1,0.02355,0.023738,0.001063,0.001129
